<a href="https://colab.research.google.com/github/Bstrato/Word2vec_document_retrieval/blob/main/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW2 Overview


In this assignment, we will study neural IR, including word embedding and LLM.

We will reuse the same Yelp dataset and refer to each individual user review as a **document**. You should reuse your JSON parser in this assignment.

The same pre-processing steps you have developed in HW1 will be used in this assignment, i.e., tokenization, stemming, normalization and stopword removal.


# Word Embedding (50 points)

Use [genism](https://radimrehurek.com/gensim/models/word2vec.html) library, download a pre-trained model or train a word2vec model (10 epochs) using the review data. Then use the model to get the vector representations of the document and query.

Hint: You can average the embedding of the terms to get the vector representation
for a document.

Use the following 10 queries (same as the ones in HW1) and retrieve the top 3 documents for each query based on cosine similarity:

	general chicken
	fried chicken
	BBQ sandwiches
	mashed potatoes
	Grilled Shrimp Salad
	lamb Shank
	Pepperoni pizza
	brussel sprout salad
	FRIENDLY STAFF
	Grilled Cheese

Note that the training does not require GPU -- moderate CPU is good enough.  If your computation power is limited, i.e., limited memory or cpu, you can choose to train the model on partial data, e.g., 50% or fewer review data. Please document the corresponding training detail in your report.

**What to submit**:

1. Paste your implementation of training document representation and cosine similarity calculation. Report the training time of the word embedding model. (15 points)
2. For the top 3 documents of each query, print the document and its cosine similarity score. (15 points)
3. For the first three queries, analyze the relation between relevance and cosine similarity score: is a high score document more relevant to the query? (10 points)
4. Are the documents more relevant than documents retrieved by TF-IDF in Homework 1? Why or why not? Compare and discuss the results. (10 points)



In [ ]:
from gensim.models import Word2Vec
import gensim.downloader

In [ ]:
pip install --upgrade gensim numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.4 MB/s eta 0:00:00


In [ ]:
import json
import nltk
import string
import os
import glob
import numpy as np
import time
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import warnings
warnings.filterwarnings('ignore')

# Try to import gensim, with fallback to manual Word2Vec implementation
try:
    from gensim.models import Word2Vec
    GENSIM_AVAILABLE = True
    print("Using Gensim Word2Vec")
except (ImportError, ValueError) as e:
    print(f"Gensim import failed: {e}")
    print("Using fallback Word2Vec implementation")
    GENSIM_AVAILABLE = False

# Try to import sklearn, with fallback to manual cosine similarity
try:
    from sklearn.metrics.pairwise import cosine_similarity
    SKLEARN_AVAILABLE = True
except ImportError:
    SKLEARN_AVAILABLE = False
    print("sklearn not available, using manual cosine similarity")

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Initialize preprocessing tools
stop_words = set(stopwords.words('english'))
punctuations = set(string.punctuation)
stemmer = PorterStemmer()


def normalize_token(token):
    """Normalize a token by converting to lowercase and handling special cases."""
    token = token.lower()
    if token in punctuations:
        return None
    if token.replace('.', '', 1).isdigit():
        return 'NUM'
    return token


def preprocess_text(text):
    """Preprocess text by tokenizing, normalizing, removing stopwords, and stemming."""
    if not text or not isinstance(text, str):
        return []

    try:
        tokens = word_tokenize(text)
        processed = []
        for token in tokens:
            norm = normalize_token(token)
            if norm is None or norm in stop_words:
                continue
            stemmed = stemmer.stem(norm)
            processed.append(stemmed)
        return processed
    except LookupError as e:
        # Fallback to simple tokenization
        simple_tokens = text.split()
        processed = []
        for token in simple_tokens:
            norm = normalize_token(token)
            if norm is None or norm in stop_words:
                continue
            stemmed = stemmer.stem(norm)
            processed.append(stemmed)
        return processed


def manual_cosine_similarity(vec1, vec2):
    """Manual implementation of cosine similarity."""
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)

    if norm1 == 0 or norm2 == 0:
        return 0.0

    return dot_product / (norm1 * norm2)


class SimpleWord2Vec:
    """
    Simplified Word2Vec implementation using skip-gram with negative sampling.
    This is a fallback when Gensim is not available.
    """

    def __init__(self, vector_size=100, window=5, min_count=2, epochs=10, learning_rate=0.025):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.vocab = {}
        self.word_vectors = {}
        self.vocab_size = 0

    def build_vocab(self, sentences):
        """Build vocabulary from sentences."""
        word_counts = {}

        # Count word frequencies
        for sentence in sentences:
            for word in sentence:
                word_counts[word] = word_counts.get(word, 0) + 1

        # Filter by min_count
        self.vocab = {word: idx for idx, (word, count) in enumerate(word_counts.items())
                     if count >= self.min_count}
        self.vocab_size = len(self.vocab)

        # Initialize random word vectors
        np.random.seed(42)
        for word in self.vocab:
            self.word_vectors[word] = np.random.uniform(-0.5, 0.5, self.vector_size)

        print(f"Built vocabulary with {self.vocab_size} words")

    def train(self, sentences):
        """Simple training using random walks."""
        self.build_vocab(sentences)

        print(f"Training for {self.epochs} epochs...")
        start_time = time.time()

        for epoch in range(self.epochs):
            if epoch % 2 == 0:
                print(f"Epoch {epoch + 1}/{self.epochs}")

            for sentence in sentences:
                sentence_words = [word for word in sentence if word in self.vocab]

                for i, target_word in enumerate(sentence_words):
                    # Get context words within window
                    start = max(0, i - self.window)
                    end = min(len(sentence_words), i + self.window + 1)

                    for j in range(start, end):
                        if i != j:
                            context_word = sentence_words[j]
                            # Simple update rule (simplified skip-gram)
                            self._update_vectors(target_word, context_word)

        training_time = time.time() - start_time
        print(f"Training completed in {training_time:.2f} seconds ({training_time/60:.2f} minutes)")
        return training_time

    def _update_vectors(self, target_word, context_word):
        """Simple vector update rule."""
        if target_word in self.word_vectors and context_word in self.word_vectors:
            # Simplified gradient update
            target_vec = self.word_vectors[target_word]
            context_vec = self.word_vectors[context_word]

            # Simple averaging update
            update = self.learning_rate * (context_vec - target_vec) * 0.1
            self.word_vectors[target_word] += update

    def get_vector(self, word):
        """Get vector for a word."""
        return self.word_vectors.get(word, np.zeros(self.vector_size))

    def __contains__(self, word):
        """Check if word is in vocabulary."""
        return word in self.word_vectors


def get_document_vector(tokens, model, vector_size=100):
    """
    Get document vector by averaging word vectors of all tokens in the document.
    Works with both Gensim and SimpleWord2Vec models.
    """
    vectors = []

    if GENSIM_AVAILABLE and hasattr(model, 'wv'):
        # Gensim model
        for token in tokens:
            if token in model.wv:
                vectors.append(model.wv[token])
    else:
        # SimpleWord2Vec model
        for token in tokens:
            if token in model:
                vectors.append(model.get_vector(token))

    if vectors:
        return np.mean(vectors, axis=0)
    else:
        # Return zero vector if no words found in vocabulary
        return np.zeros(vector_size)


def calculate_cosine_similarity(vec1, vec2):
    """Calculate cosine similarity between two vectors."""
    if SKLEARN_AVAILABLE:
        # Use sklearn implementation
        vec1 = vec1.reshape(1, -1)
        vec2 = vec2.reshape(1, -1)
        return cosine_similarity(vec1, vec2)[0][0]
    else:
        # Use manual implementation
        return manual_cosine_similarity(vec1, vec2)


def word2vec_document_retrieval(yelp_folder="/content/sample_data/yelp"):
    """
    Main function to train Word2Vec model and perform document retrieval.
    """
    json_files = glob.glob(os.path.join(yelp_folder, "*.json"))

    if not json_files:
        print(f"No JSON files found in {yelp_folder}")
        return

    print(f"Found {len(json_files)} JSON files in {yelp_folder}")

    # Data structures for processing
    documents = []  # Store original documents with metadata
    processed_documents = []  # Store processed tokens for each document
    total_reviews = 0

    # Process all JSON files
    for file_path in json_files:
        print(f"\nProcessing file: {os.path.basename(file_path)}")

        try:
            with open(file_path, 'r') as f:
                try:
                    data = json.load(f)
                except json.JSONDecodeError:
                    f.seek(0)
                    data = []
                    for line in f:
                        if line.strip():
                            data.append(json.loads(line.strip()))

            # Extract reviews
            reviews = []
            if isinstance(data, dict) and 'Reviews' in data:
                reviews = data['Reviews']
                print(f"Found {len(reviews)} reviews in a single JSON object")
            elif isinstance(data, list):
                for obj in data:
                    if isinstance(obj, dict) and 'Reviews' in obj:
                        reviews.extend(obj['Reviews'])
                print(f"Found {len(reviews)} total reviews across multiple objects")

            if not reviews:
                print("No reviews found in this file!")
                continue

            # Find review text field and review ID field
            potential_text_fields = ['Content', 'ReviewText', 'Text', 'review_text', 'comment', 'content', 'review']
            potential_id_fields = ['ReviewID', 'ReviewId', 'review_id', 'id', 'Id', 'ID', 'reviewId', 'Review_ID']

            review_field_name = None
            review_id_field = None

            # Find review text field
            for field in potential_text_fields:
                if field in reviews[0]:
                    review_field_name = field
                    print(f"Found review text in field: '{field}'")
                    break

            # Find review ID field
            for field in potential_id_fields:
                if field in reviews[0]:
                    review_id_field = field
                    print(f"Found review ID in field: '{field}'")
                    break

            if not review_field_name:
                print("Could not identify review text field. Available fields:")
                print(list(reviews[0].keys()) if reviews else "No reviews found")
                continue

            if not review_id_field:
                print("Could not identify review ID field. Will use generated IDs.")
                print("Available fields:", list(reviews[0].keys()) if reviews else "No reviews found")

            # Process each review
            for i, review in enumerate(reviews):
                total_reviews += 1
                text = review.get(review_field_name, '')
                tokens = preprocess_text(text)

                if tokens:  # Only add non-empty documents
                    processed_documents.append(tokens)

                    # Use actual review ID if available, otherwise generate one
                    if review_id_field and review.get(review_id_field):
                        review_id = str(review.get(review_id_field))
                    else:
                        review_id = f"{os.path.basename(file_path)}_review_{i}"

                    documents.append({
                        'id': review_id,
                        'text': text,
                        'tokens': tokens,
                        'file': os.path.basename(file_path)
                    })

        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

    print(f"\nProcessed {total_reviews} reviews in total")
    print(f"Valid documents for retrieval: {len(documents)}")

    # Train Word2Vec model
    print("\n" + "="*50)
    print("TRAINING WORD2VEC MODEL")
    print("="*50)

    print(f"Training Word2Vec model with {len(processed_documents)} documents...")

    training_start_time = time.time()

    if GENSIM_AVAILABLE:
        # Use Gensim Word2Vec
        try:
            model = Word2Vec(
                sentences=processed_documents,
                vector_size=100,
                window=5,
                min_count=2,
                workers=4,
                epochs=10,
                sg=0  # CBOW algorithm
            )
            training_time = time.time() - training_start_time
            print(f"Gensim Word2Vec model trained successfully!")
            print(f"Training time: {training_time:.2f} seconds ({training_time/60:.2f} minutes)")
            print(f"Vocabulary size: {len(model.wv.key_to_index)}")
        except Exception as e:
            print(f"Gensim training failed: {e}")
            print("Falling back to simple Word2Vec implementation...")
            model = SimpleWord2Vec(vector_size=100, window=5, min_count=2, epochs=10)
            training_time = model.train(processed_documents)
            print(f"Simple Word2Vec model trained successfully!")
            print(f"Vocabulary size: {model.vocab_size}")
    else:
        # Use fallback implementation
        model = SimpleWord2Vec(vector_size=100, window=5, min_count=2, epochs=10)
        training_time = model.train(processed_documents)
        print(f"Simple Word2Vec model trained successfully!")
        print(f"Vocabulary size: {model.vocab_size}")

    # Generate document vectors
    print("\nGenerating document vectors...")
    document_vectors = []
    for doc in documents:
        doc_vector = get_document_vector(doc['tokens'], model, vector_size=100)
        document_vectors.append(doc_vector)

    print(f"Generated {len(document_vectors)} document vectors")

    # Document Retrieval
    print("\n" + "="*50)
    print("DOCUMENT RETRIEVAL")
    print("="*50)

    queries = [
        "general chicken",
        "fried chicken",
        "BBQ sandwiches",
        "mashed potatoes",
        "Grilled Shrimp Salad",
        "lamb Shank",
        "Pepperoni pizza",
        "brussel sprout salad",
        "FRIENDLY STAFF",
        "Grilled Cheese"
    ]

    all_results = {}

    for query in queries:
        print(f"\nQuery: '{query}'")
        print("-" * 40)

        # Preprocess query
        query_tokens = preprocess_text(query)
        print(f"Processed query tokens: {query_tokens}")

        # Get query vector
        query_vector = get_document_vector(query_tokens, model, vector_size=100)

        # Calculate similarities with all documents
        similarities = []
        for i, doc_vector in enumerate(document_vectors):
            similarity = calculate_cosine_similarity(query_vector, doc_vector)
            similarities.append((i, similarity))

        # Sort by similarity (descending)
        similarities.sort(key=lambda x: x[1], reverse=True)

        # Get top 3 results
        top_3 = similarities[:3]
        all_results[query] = []

        print("Top 3 most similar documents:")
        for rank, (doc_idx, similarity) in enumerate(top_3, 1):
            doc = documents[doc_idx]
            print(f"  {rank}. Similarity: {similarity:.4f}")
            print(f"     Review ID: {doc['id']}")
            print(f"     Text preview: {doc['text'][:200]}...")
            print()

            all_results[query].append({
                'rank': rank,
                'review_id': doc['id'],
                'similarity': similarity,
                'text_preview': doc['text'][:200]
            })

    # Summary of results
    print("\n" + "="*50)
    print("RETRIEVAL RESULTS SUMMARY")
    print("="*50)

    for query, results in all_results.items():
        print(f"\nQuery: '{query}'")
        for result in results:
            print(f"  {result['rank']}. {result['review_id']} (sim: {result['similarity']:.4f})")

    # Save Word2Vec model
    try:
        if GENSIM_AVAILABLE and hasattr(model, 'save'):
            model.save("word2vec_model.model")
            print(f"\nGensim Word2Vec model saved as 'word2vec_model.model'")
        else:
            # Save simple model using pickle or numpy
            import pickle
            with open("simple_word2vec_model.pkl", "wb") as f:
                pickle.dump(model, f)
            print(f"\nSimple Word2Vec model saved as 'simple_word2vec_model.pkl'")
    except Exception as e:
        print(f"Could not save model: {e}")

    return {
        'total_reviews': total_reviews,
        'total_documents': len(documents),
        'vocabulary_size': len(model.wv.key_to_index) if GENSIM_AVAILABLE and hasattr(model, 'wv') else model.vocab_size,
        'training_time_seconds': training_time,
        'training_time_minutes': training_time/60,
        'retrieval_results': all_results,
        'model': model
    }


if __name__ == "__main__":
    results = word2vec_document_retrieval()

Gensim import failed: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject
Using fallback Word2Vec implementation
Found 60 JSON files in /content/sample_data/yelp

Processing file: gQCn4Gv-4_UUUFwpo-zHvA.json
Found 1848 reviews in a single JSON object
Found review text in field: 'Content'
Found review ID in field: 'ReviewID'


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Processing file: 7lnOgOMd7zys3bFfWY_jIw.json
Found 2171 reviews in a single JSON object
Found review text in field: 'Content'
Found review ID in field: 'ReviewID'

Processing file: WU_dFObt9VxHMcS1Eu32iQ.json
Found 2664 reviews in a single JSON object
Found review text in field: 'Content'
Found review ID in field: 'ReviewID'

Processing file: dlDEuDIvZI6I0cGZy4jIYg.json
Found 1045 reviews in a single JSON object
Found review text in field: 'Content'
Found review ID in field: 'ReviewID'

Processing file: J_rNhtb144k_fW50UQ7_lg.json
Found 1511 reviews in a single JSON object
Found review text in field: 'Content'
Found review ID in field: 'ReviewID'

Processing file: p8Uu-CYOUaISZT4w6OTNrQ.json
Found 1583 reviews in a single JSON object
Found review text in field: 'Content'
Found review ID in field: 'ReviewID'

Processing file: 3OLZOlqgOXdqY0uwxcOTfw.json
Found 1951 reviews in a single JSON object
Found review text in field: 'Content'
Found review ID in field: 'ReviewID'

Processing file

## Retrieval with Large Language Models (LLMs) (40 points)

This task introduces using LLMs for document retrieval through the free OpenRouter API. You will explore how to prompt an LLM to find relevant documents from the Yelp review dataset.

**Setup:**
1.  Sign up for a free account at [OpenRouter.ai](https://openrouter.ai/).
2.  Obtain your API key from your OpenRouter dashboard.
3.  Familiarize yourself with the [OpenRouter API Quickstart Guide](https://openrouter.ai/docs/quickstart), particularly how to make API calls using Python (the `openai` library is often used as OpenRouter mimics its API structure for many models).
4.  Choose one of the free models available on OpenRouter for your experiments (e.g., Mistral 7B Instruct, Qwen 8B, etc.). Note the model you choose in your report.

**Activities:**

1.  **Introduction to LLM-based Retrieval (15 points)**
    * Briefly explain (1 paragraph in your report) the concept of using LLMs for retrieval. What are potential advantages and disadvantages compared to traditional methods (TF-IDF) and embedding-based methods (Word2Vec/Doc2Vec)?
    * Include a small code snippet that shows you can successfully connect to the OpenRouter API with your chosen model (e.g., a simple test query like "What is the capital of France?").

2.  **Prompt Engineering for Document Retrieval (20 points)**
    * Use the same 10 queries from previous tasks.
    * **Strategy for presenting documents to the LLM:** Since LLM context windows are limited, you cannot pass all reviews for every query. You should devise a strategy. For example:
        * **Option A (Re-ranking):** First, retrieve a larger set of candidate documents (e.g., top 5-10) for each query using one of your previous methods (TF-IDF, Word2Vec). Then, for each query, present these candidate documents (e.g., their content) to the LLM and ask it to select or re-rank the top 3 most relevant ones based on the query.
        * **Option B (Direct Retrieval from a Subset):** For each query, randomly sample a small subset of reviews from the entire dataset. Present these to the LLM and ask it to identify the top 3 relevant to the query from this subset. (Repeat sampling if no relevant docs are found, or acknowledge this limitation).
        * *Clearly describe the strategy you choose and why.*
    * **Prompt Design:** Design 2-3 different prompt strategies to instruct the LLM. Examples:
        * *Zero-shot Re-ranking:* "Given the query '{query}' and the following reviews, which 3 reviews are most relevant? Provide only their IDs/numbers.
Review 1: {review_content_1}
Review 2: {review_content_2}
..."
        * *Instruction-based Selection:* "You are a helpful assistant. I am looking for restaurant reviews about '{query}'. From the list of reviews below, please identify the top 3 reviews that best match this topic. Focus on {specific aspect, e.g., food quality, specific dish mentioned}. Return the review text or ID.
Reviews:
{document_bundle}"
    * For each query, apply your chosen document presentation strategy and your *best performing* prompt strategy to retrieve the top 3 documents (reviews).
    * **Deliverables for this part:**
        * Clear documentation of your chosen document presentation strategy (Option A/B or other).
        * The exact text of the 2-3 prompt strategies you designed.
        * Python code snippets showing how you interacted with the OpenRouter API (formatting the prompt with the query and document data, making the call, parsing the response).
        * The top 3 retrieved document IDs (or snippets) for each of the 10 queries using your best prompt.
3. **Discussion and analysis (5 points)**
    * Compare these LLM-retrieved results against those obtained from TF-IDF and Word2Vec. Which of your prompt strategies seemed to work best? Why do you think so?
    * Discuss any challenges you faced (e.g., prompt sensitivity, LLM hallucinations, managing context length, API errors, cost/rate limits if applicable even on free tiers, parsing LLM output). What are the pros and cons of using LLMs for this retrieval task based on your experience?

**What to submit:**

* Your written explanation for the first question.
* Your documented document presentation strategy, prompt designs, code for API interaction, and the top 3 retrieved documents per query.
* Your discussion and analysis for question 3.

In [ ]:
import openai
import os
import json # For handling review data


print("Task 3: Retrieval with LLMs via OpenRouter")
# --- Your code for Task 3 will go here ---

# **Important:** Set your OpenRouter API key as an environment variable
# or directly in the code (less secure, for testing only).
# os.environ['OPENROUTER_API_KEY'] = "YOUR_OPENROUTER_API_KEY"
# client = openai.OpenAI(
#     base_url="https://openrouter.ai/api/v1",
#     api_key=os.getenv("OPENROUTER_API_KEY"),
# )

# Example: Test API connection
# CHOSEN_LLM_MODEL = "mistralai/mistral-7b-instruct:free" # Example free model
# try:
#     completion = client.chat.completions.create(
#         model=CHOSEN_LLM_MODEL,
#         messages=[
#             {"role": "system", "content": "You are a helpful assistant."},
#             {"role": "user", "content": "What is the capital of France?"},
#         ],
#         max_tokens=10,
#         temperature=0.1
#     )
#     print(completion.choices[0].message.content)
# except Exception as e:
#     print(f"Error connecting to OpenRouter: {e}")



In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.5/725.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 25.9 MB/s eta 0:00:00


In [ ]:
import openai
import os
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import time
import re
import glob
import string
import nltk
import warnings
from typing import List, Dict, Tuple
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

warnings.filterwarnings('ignore')

# Download required NLTK data
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
except:
    print("Note: NLTK downloads may have failed, using fallback tokenization")

# Initialize preprocessing tools
try:
    stop_words = set(stopwords.words('english'))
except:
    stop_words = {'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours'}

punctuations = set(string.punctuation)
stemmer = PorterStemmer()

def normalize_token(token):
    """Normalize a token by converting to lowercase and handling special cases."""
    token = token.lower()
    if token in punctuations:
        return None
    if token.replace('.', '', 1).isdigit():
        return 'NUM'
    return token

def preprocess_text(text):
    """Preprocess text by tokenizing, normalizing, removing stopwords, and stemming."""
    if not text or not isinstance(text, str):
        return []

    try:
        tokens = word_tokenize(text)
        processed = []
        for token in tokens:
            norm = normalize_token(token)
            if norm is None or norm in stop_words:
                continue
            stemmed = stemmer.stem(norm)
            processed.append(stemmed)
        return processed
    except LookupError:
        # Fallback to simple tokenization if NLTK data not available
        simple_tokens = text.split()
        processed = []
        for token in simple_tokens:
            norm = normalize_token(token)
            if norm is None or norm in stop_words:
                continue
            stemmed = stemmer.stem(norm)
            processed.append(stemmed)
        return processed

print("Task 3: Retrieval with LLMs via OpenRouter")

# **IMPORTANT:** Set your OpenRouter API key here
# Replace "YOUR_API_KEY_HERE" with your actual OpenRouter API key
os.environ['OPENROUTER_API_KEY'] = "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"

# Initialize OpenRouter client
client = openai.OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

# Choose a free model
CHOSEN_LLM_MODEL = "qwen/qwen3-8b:free"

# Test API connection first
def test_api_connection():
    """Test the OpenRouter API connection"""
    print("\n=== Testing API Connection ===")
    try:
        completion = client.chat.completions.create(
            model=CHOSEN_LLM_MODEL,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "What is the capital of France?"},
            ],
            max_tokens=10,
            temperature=0.1
        )
        print(f"✓ API Connection Successful!")
        print(f"Test Response: {completion.choices[0].message.content}")
        return True
    except Exception as e:
        print(f"✗ Error connecting to OpenRouter: {e}")
        return False

# Load Yelp reviews data
def find_yelp_files(directory_path: str) -> List[str]:
    """Find Yelp data files in the directory"""

    print(f"\n=== Scanning directory: {directory_path} ===")

    # Common Yelp file patterns
    patterns = [
        "yelp_academic_dataset_review.json",
        "review.json",
        "*review*.json",
        "*.json"
    ]

    found_files = []
    for pattern in patterns:
        full_pattern = os.path.join(directory_path, pattern)
        matches = glob.glob(full_pattern)
        if matches:
            found_files.extend(matches)
            break  # Use first successful pattern

    if found_files:
        print(f"Found files: {found_files}")
    else:
        print(f"No JSON files found in {directory_path}")
        # List all files in directory for debugging
        try:
            all_files = os.listdir(directory_path)
            print(f"All files in directory: {all_files}")
        except:
            pass

    return found_files

def load_yelp_data(file_path: str) -> List[Dict]:
    """Load Yelp reviews from JSON file with robust parsing"""
    print(f"\n=== Loading Yelp Data from {file_path} ===")

    # If path is a directory, try to find the review file
    if os.path.isdir(file_path):
        print(f"Path is a directory. Searching for review files...")
        found_files = find_yelp_files(file_path)
        if not found_files:
            return []
        file_path = found_files[0]  # Use first found file
        print(f"Using file: {file_path}")

    try:
        documents = []
        total_reviews = 0

        with open(file_path, 'r', encoding='utf-8') as f:
            # Try to parse as single JSON first
            try:
                f.seek(0)
                data = json.load(f)

                # Handle different JSON structures
                reviews = []
                if isinstance(data, dict) and 'Reviews' in data:
                    reviews = data['Reviews']
                    print(f"Found {len(reviews)} reviews in a single JSON object")
                elif isinstance(data, list):
                    for obj in data:
                        if isinstance(obj, dict):
                            if 'Reviews' in obj:
                                reviews.extend(obj['Reviews'])
                            elif 'text' in obj or 'review_text' in obj:  # Direct review objects
                                reviews.append(obj)
                    print(f"Found {len(reviews)} total reviews")
                elif isinstance(data, dict) and ('text' in data or 'review_text' in data):
                    reviews = [data]  # Single review object

                if not reviews:
                    print("No reviews found in JSON structure!")
                    return []

                # Process reviews with flexible field detection
                potential_text_fields = ['text', 'Content', 'ReviewText', 'Text', 'review_text', 'comment', 'content', 'review']
                potential_id_fields = ['review_id', 'ReviewID', 'ReviewId', 'id', 'Id', 'ID', 'reviewId', 'Review_ID']

                review_field_name = None
                review_id_field = None

                # Find review text field
                for field in potential_text_fields:
                    if field in reviews[0]:
                        review_field_name = field
                        print(f"Found review text in field: '{field}'")
                        break

                # Find review ID field
                for field in potential_id_fields:
                    if field in reviews[0]:
                        review_id_field = field
                        print(f"Found review ID in field: '{field}'")
                        break

                if not review_field_name:
                    print("Could not identify review text field. Available fields:")
                    print(list(reviews[0].keys()) if reviews else "No reviews found")
                    return []

                # Process each review
                for i, review in enumerate(reviews):
                    if i >= 1000:  # Limit for efficiency
                        break

                    total_reviews += 1
                    text = review.get(review_field_name, '')

                    if not text or not isinstance(text, str):
                        continue

                    # Preprocess the text
                    processed_tokens = preprocess_text(text)

                    if processed_tokens:  # Only add non-empty documents
                        # Use actual review ID if available, otherwise generate one
                        if review_id_field and review.get(review_id_field):
                            review_id = str(review.get(review_id_field))
                        else:
                            review_id = f"review_{i}"

                        documents.append({
                            'review_id': review_id,
                            'text': text,
                            'processed_text': ' '.join(processed_tokens),  # For TF-IDF
                            'tokens': processed_tokens,
                            'stars': review.get('stars', 0),
                            'business_id': review.get('business_id', '')
                        })

            except json.JSONDecodeError:
                # Fallback to line-by-line parsing (JSONL format)
                print("Single JSON parsing failed, trying line-by-line parsing...")
                f.seek(0)
                for line_num, line in enumerate(f):
                    if line_num >= 1000:  # Limit for efficiency
                        break
                    try:
                        line = line.strip()
                        if not line:
                            continue
                        review = json.loads(line)

                        # Extract text
                        text = review.get('text', '')
                        if not text:
                            continue

                        # Preprocess the text
                        processed_tokens = preprocess_text(text)

                        if processed_tokens:
                            documents.append({
                                'review_id': review.get('review_id', f'review_{line_num}'),
                                'text': text,
                                'processed_text': ' '.join(processed_tokens),
                                'tokens': processed_tokens,
                                'stars': review.get('stars', 0),
                                'business_id': review.get('business_id', '')
                            })
                        total_reviews += 1

                    except json.JSONDecodeError:
                        continue
                    except Exception as e:
                        print(f"Skipping line {line_num}: {e}")
                        continue

        print(f"✓ Processed {total_reviews} reviews")
        print(f"✓ Loaded {len(documents)} valid documents with preprocessing")
        return documents

    except FileNotFoundError:
        print(f"✗ File not found: {file_path}")
        return []
    except Exception as e:
        print(f"✗ Error loading data: {e}")
        return []

class LLMRetriever:
    """LLM-based document retrieval system using OpenRouter"""

    def __init__(self, api_key: str = None, model_name: str = CHOSEN_LLM_MODEL):
        self.client = client
        self.model_name = model_name
        self.request_delay = 1  # Delay between API calls to avoid rate limits

    def get_candidates_tfidf(self, query: str, documents: List[Dict], top_k: int = 5) -> Tuple[List[str], List[str]]:
        """Get candidate documents using TF-IDF for initial filtering with preprocessing"""
        print(f"Getting TF-IDF candidates for query: '{query}'")

        # Preprocess the query
        query_tokens = preprocess_text(query)
        processed_query = ' '.join(query_tokens)
        print(f"Processed query: '{processed_query}'")

        # Extract preprocessed texts
        processed_texts = [doc['processed_text'] for doc in documents]
        doc_ids = [doc['review_id'] for doc in documents]

        # Create TF-IDF vectors using preprocessed text
        vectorizer = TfidfVectorizer(
            stop_words=None,  # We already removed stopwords during preprocessing
            max_features=1000,
            ngram_range=(1, 2),
            min_df=2,
            lowercase=False  # Already lowercased during preprocessing
        )

        try:
            doc_vectors = vectorizer.fit_transform(processed_texts)
            query_vector = vectorizer.transform([processed_query])

            # Calculate similarities
            similarities = cosine_similarity(query_vector, doc_vectors).flatten()
            top_indices = np.argsort(similarities)[::-1][:top_k]

            candidate_docs = [documents[i]['text'] for i in top_indices]  # Use original text for LLM
            candidate_ids = [doc_ids[i] for i in top_indices]

            print(f"Selected {len(candidate_docs)} candidates")
            top_sims = [f"{similarities[i]:.4f}" for i in top_indices[:3]]
            print(f"Top similarities: {top_sims}")
            return candidate_ids, candidate_docs

        except Exception as e:
            print(f"TF-IDF error: {e}, falling back to first {top_k} documents")
            return doc_ids[:top_k], [doc['text'] for doc in documents[:top_k]]

    def llm_rerank(self, query: str, candidate_docs: List[str], candidate_ids: List[str],
                   prompt_strategy: str = "zero_shot") -> List[str]:
        """Use LLM to re-rank candidate documents"""

        print(f"Re-ranking with LLM using '{prompt_strategy}' strategy")

        # Build prompt based on strategy
        if prompt_strategy == "zero_shot":
            prompt = self._build_zero_shot_prompt(query, candidate_docs, candidate_ids)
        elif prompt_strategy == "instruction_based":
            prompt = self._build_instruction_prompt(query, candidate_docs, candidate_ids)
        elif prompt_strategy == "detailed_analysis":
            prompt = self._build_detailed_prompt(query, candidate_docs, candidate_ids)
        else:
            raise ValueError(f"Unknown prompt strategy: {prompt_strategy}")

        try:
            # Add delay to avoid rate limits
            time.sleep(self.request_delay)

            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=300,
                temperature=0.1  # Low temperature for consistency
            )

            llm_response = response.choices[0].message.content
            print(f"LLM Response: {llm_response[:100]}...")

            # Parse the response to extract selected document IDs
            selected_ids = self._parse_llm_response(llm_response, candidate_ids)
            return selected_ids

        except Exception as e:
            print(f"LLM API Error: {e}")
            # Fallback to TF-IDF ranking
            return candidate_ids[:3]

    def _build_zero_shot_prompt(self, query: str, docs: List[str], ids: List[str]) -> str:
        """Build zero-shot re-ranking prompt"""
        doc_text = ""
        for i, (doc, doc_id) in enumerate(zip(docs, ids)):
            # Truncate long reviews to fit context window
            truncated_doc = doc[:300] + "..." if len(doc) > 300 else doc
            doc_text += f"Review {i+1} (ID: {doc_id}): {truncated_doc}\n\n"

        return f"""Given the query '{query}' and the following restaurant reviews, which 3 reviews are most relevant to the query?

Provide only the review numbers (1, 2, 3, etc.) in order of relevance, separated by commas.

{doc_text}

Most relevant reviews (numbers only):"""

    def _build_instruction_prompt(self, query: str, docs: List[str], ids: List[str]) -> str:
        """Build instruction-based prompt"""
        doc_text = ""
        for i, (doc, doc_id) in enumerate(zip(docs, ids)):
            truncated_doc = doc[:300] + "..." if len(doc) > 300 else doc
            doc_text += f"Review {i+1} (ID: {doc_id}): {truncated_doc}\n\n"

        return f"""You are a helpful assistant specializing in restaurant review analysis.
I am searching for restaurant reviews about: '{query}'

From the reviews below, identify the top 3 reviews that best match this search query.
Focus on reviews that specifically mention or relate to the query topic.
Consider both direct mentions and contextual relevance.

Return only the review numbers (1, 2, 3, etc.) in order of relevance, separated by commas.

Reviews:
{doc_text}

Top 3 most relevant reviews (numbers only):"""

    def _build_detailed_prompt(self, query: str, docs: List[str], ids: List[str]) -> str:
        """Build detailed analysis prompt"""
        doc_text = ""
        for i, (doc, doc_id) in enumerate(zip(docs, ids)):
            truncated_doc = doc[:300] + "..." if len(doc) > 300 else doc
            doc_text += f"Review {i+1} (ID: {doc_id}): {truncated_doc}\n\n"

        return f"""Analyze the following restaurant reviews to find the 3 most relevant ones for the query: '{query}'

Consider these factors when ranking:
1. Direct mentions of the query topic
2. Contextual relevance and semantic similarity
3. Quality and detail of information related to the query
4. Overall helpfulness for someone searching for '{query}'

Reviews:
{doc_text}

Based on your analysis, provide the top 3 review numbers in order of relevance (numbers only, separated by commas):"""

    def _parse_llm_response(self, response: str, candidate_ids: List[str]) -> List[str]:
        """Parse LLM response to extract document IDs"""
        # Look for numbers in the response
        numbers = re.findall(r'\b([1-9])\b', response)

        selected_ids = []
        for num_str in numbers[:3]:  # Take only first 3
            try:
                idx = int(num_str) - 1  # Convert to 0-based index
                if 0 <= idx < len(candidate_ids):
                    selected_ids.append(candidate_ids[idx])
            except ValueError:
                continue

        # If we don't have enough, pad with remaining candidates
        while len(selected_ids) < 3 and len(selected_ids) < len(candidate_ids):
            for cid in candidate_ids:
                if cid not in selected_ids:
                    selected_ids.append(cid)
                    break

        return selected_ids[:3]

def run_llm_retrieval_experiment():
    """Main function to run the LLM retrieval experiment"""

    # Test API connection first
    if not test_api_connection():
        print("Cannot proceed without API connection. Please check your API key.")
        return

    # Load Yelp data - now handles directory paths and includes preprocessing
    reviews_data = load_yelp_data("/content/sample_data/yelp")

    if not reviews_data:
        print("No data loaded. Cannot proceed with experiment.")
        return

    print(f"Loaded {len(reviews_data)} preprocessed documents")

    # Define test queries (use the same 10 queries from previous tasks)
    test_queries = [
        "general chicken",
        "fried chicken",
        "BBQ sandwiches",
        "mashed potatoes",
        "Grilled Shrimp Salad",
        "lamb Shank",
        "Pepperoni pizza",
        "brussel sprout salad",
        "FRIENDLY STAFF",
        "Grilled Cheese"
    ]

    # Initialize retriever
    retriever = LLMRetriever()

    # Test different prompt strategies
    prompt_strategies = ["zero_shot", "instruction_based", "detailed_analysis"]

    results = {}

    print(f"\n=== Running LLM Retrieval Experiment ===")
    print(f"Testing {len(test_queries)} queries with {len(prompt_strategies)} prompt strategies")

    for query in test_queries:
        print(f"\n--- Processing Query: '{query}' ---")
        results[query] = {}

        # Get TF-IDF candidates first (now uses preprocessed documents)
        candidate_ids, candidate_docs = retriever.get_candidates_tfidf(
            query, reviews_data, top_k=5
        )

        # Test each prompt strategy
        for strategy in prompt_strategies:
            print(f"\nTesting strategy: {strategy}")
            try:
                selected_ids = retriever.llm_rerank(
                    query, candidate_docs, candidate_ids, strategy
                )
                results[query][strategy] = selected_ids
                print(f"Selected documents: {selected_ids}")
            except Exception as e:
                print(f"Error with strategy {strategy}: {e}")
                results[query][strategy] = candidate_ids[:3]  # Fallback

    return results, reviews_data

def analyze_and_report_results(results: Dict, reviews_data: List[Dict]):
    """Analyze and report the LLM retrieval results with document details"""
    print(f"\n=== LLM Retrieval Results Analysis ===")

    # Report results for each query with document previews
    for query, strategies in results.items():
        print(f"\nQuery: '{query}'")
        print("-" * 50)

        # Find the documents for better analysis
        doc_lookup = {doc['review_id']: doc for doc in reviews_data}

        for strategy, doc_ids in strategies.items():
            print(f"\n{strategy.upper()} Results:")
            for i, doc_id in enumerate(doc_ids, 1):
                if doc_id in doc_lookup:
                    doc = doc_lookup[doc_id]
                    text_preview = doc['text'][:150] + "..." if len(doc['text']) > 150 else doc['text']
                    print(f"  {i}. ID: {doc_id}")
                    print(f"     Preview: {text_preview}")
                    print(f"     Stars: {doc.get('stars', 'N/A')}")
                else:
                    print(f"  {i}. ID: {doc_id} (document not found)")

    # Calculate strategy consistency
    print(f"\n=== Strategy Consistency Analysis ===")
    for query in results:
        strategies = list(results[query].keys())
        if len(strategies) >= 2:
            overlap_counts = {}
            for i, strategy1 in enumerate(strategies):
                for strategy2 in strategies[i+1:]:
                    docs1 = set(results[query][strategy1])
                    docs2 = set(results[query][strategy2])
                    overlap = len(docs1.intersection(docs2))
                    pair = f"{strategy1} vs {strategy2}"
                    if pair not in overlap_counts:
                        overlap_counts[pair] = []
                    overlap_counts[pair].append(overlap)

            print(f"\nQuery '{query}' - Strategy overlaps:")
            for pair, overlaps in overlap_counts.items():
                avg_overlap = sum(overlaps) / len(overlaps)
                print(f"  {pair}: {avg_overlap:.1f}/3 documents overlap")

# Part 1: Conceptual explanation (already covered in the guide)
print("\n=== Part 1: Introduction to LLM-based Retrieval ===")
print("""
LLM-based Retrieval leverages large language models' natural language understanding
to identify relevant documents based on semantic meaning rather than just keyword matching.

Advantages over traditional methods:
- Better semantic understanding and context awareness
- Can handle complex, conversational queries
- Understands synonyms, paraphrases, and implicit meanings

Disadvantages:
- Higher computational cost and slower processing
- Limited by context window size
- Less consistent due to probabilistic nature
- Risk of hallucination in relevance judgments

""")

# Part 2: Run the experiment
if __name__ == "__main__":
    # Run the main experiment
    experiment_results, reviews_data = run_llm_retrieval_experiment()

    if experiment_results:
        # Analyze and report results
        analyze_and_report_results(experiment_results, reviews_data)

        # Save results to file with additional metadata
        output_data = {
            'experiment_results': experiment_results,
            'metadata': {
                'total_documents': len(reviews_data),
                'queries_tested': len(experiment_results),
                'strategies_used': ['zero_shot', 'instruction_based', 'detailed_analysis'],
                'preprocessing_applied': True,
                'model_used': CHOSEN_LLM_MODEL
            }
        }

        with open('llm_retrieval_results.json', 'w') as f:
            json.dump(output_data, f, indent=2)
        print(f"\nResults saved to 'llm_retrieval_results.json'")

        print(f"\n=== Experiment Complete ===")
        print(f"✓ Tested {len(experiment_results)} queries")
        print(f"✓ Used 3 different prompt strategies")
        print(f"✓ Applied consistent preprocessing from previous tasks")
        print(f"✓ Implemented re-ranking approach with TF-IDF candidates")
    else:
        print("Experiment failed. Please check your setup and try again.")

Task 3: Retrieval with LLMs via OpenRouter

=== Part 1: Introduction to LLM-based Retrieval ===

LLM-based Retrieval leverages large language models' natural language understanding 
to identify relevant documents based on semantic meaning rather than just keyword matching.

Advantages over traditional methods:
- Better semantic understanding and context awareness
- Can handle complex, conversational queries
- Understands synonyms, paraphrases, and implicit meanings

Disadvantages:
- Higher computational cost and slower processing
- Limited by context window size
- Less consistent due to probabilistic nature
- Risk of hallucination in relevance judgments

Chosen model: mistralai/mistral-7b-instruct:free


=== Testing API Connection ===
✓ API Connection Successful!
Test Response: 

=== Loading Yelp Data from /content/sample_data/yelp ===
Path is a directory. Searching for review files...

=== Scanning directory: /content/sample_data/yelp ===
Found files: ['/content/sample_data/yelp/dlDEu

Implementing pointwise, pairwise, and/or listwise prompts. Retrieving the top 3 documents for each query.

In [ ]:
import openai
import os
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import time
import re
import glob
import string
import nltk
import warnings
from typing import List, Dict, Tuple
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import itertools

warnings.filterwarnings('ignore')

# Download required NLTK data
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
except:
    print("Note: NLTK downloads may have failed, using fallback tokenization")

# Initialize preprocessing tools
try:
    stop_words = set(stopwords.words('english'))
except:
    stop_words = {'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours'}

punctuations = set(string.punctuation)
stemmer = PorterStemmer()

def normalize_token(token):
    """Normalize a token by converting to lowercase and handling special cases."""
    token = token.lower()
    if token in punctuations:
        return None
    if token.replace('.', '', 1).isdigit():
        return 'NUM'
    return token

def preprocess_text(text):
    """Preprocess text by tokenizing, normalizing, removing stopwords, and stemming."""
    if not text or not isinstance(text, str):
        return []

    try:
        tokens = word_tokenize(text)
        processed = []
        for token in tokens:
            norm = normalize_token(token)
            if norm is None or norm in stop_words:
                continue
            stemmed = stemmer.stem(norm)
            processed.append(stemmed)
        return processed
    except LookupError:
        simple_tokens = text.split()
        processed = []
        for token in simple_tokens:
            norm = normalize_token(token)
            if norm is None or norm in stop_words:
                continue
            stemmed = stemmer.stem(norm)
            processed.append(stemmed)
        return processed

print("Advanced LLM Retrieval: Pointwise, Pairwise, and Listwise Approaches")

# **IMPORTANT:** Set your OpenRouter API key here
os.environ['OPENROUTER_API_KEY'] = "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"

# Initialize OpenRouter client
client = openai.OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

# Choose a free model
CHOSEN_LLM_MODEL = "qwen/qwen3-8b:free"

def test_api_connection():
    """Test the OpenRouter API connection"""
    print("\n=== Testing API Connection ===")
    try:
        completion = client.chat.completions.create(
            model=CHOSEN_LLM_MODEL,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "What is the capital of France?"},
            ],
            max_tokens=10,
            temperature=0.1
        )
        print(f"✓ API Connection Successful!")
        print(f"Test Response: {completion.choices[0].message.content}")
        return True
    except Exception as e:
        print(f"✗ Error connecting to OpenRouter: {e}")
        return False

def find_yelp_files(directory_path: str) -> List[str]:
    """Find Yelp data files in the directory"""
    print(f"\n=== Scanning directory: {directory_path} ===")

    patterns = [
        "yelp_academic_dataset_review.json",
        "review.json",
        "*review*.json",
        "*.json"
    ]

    found_files = []
    for pattern in patterns:
        full_pattern = os.path.join(directory_path, pattern)
        matches = glob.glob(full_pattern)
        if matches:
            found_files.extend(matches)
            break

    if found_files:
        print(f"Found files: {found_files[:3]}...")  # Show first 3
    else:
        print(f"No JSON files found in {directory_path}")
        try:
            all_files = os.listdir(directory_path)
            print(f"All files in directory: {all_files}")
        except:
            pass

    return found_files

def load_yelp_data(file_path: str) -> List[Dict]:
    """Load Yelp reviews from JSON file with robust parsing"""
    print(f"\n=== Loading Yelp Data from {file_path} ===")

    if os.path.isdir(file_path):
        print(f"Path is a directory. Searching for review files...")
        found_files = find_yelp_files(file_path)
        if not found_files:
            return []
        file_path = found_files[0]
        print(f"Using file: {file_path}")

    try:
        documents = []
        total_reviews = 0

        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                f.seek(0)
                data = json.load(f)

                reviews = []
                if isinstance(data, dict) and 'Reviews' in data:
                    reviews = data['Reviews']
                    print(f"Found {len(reviews)} reviews in a single JSON object")
                elif isinstance(data, list):
                    for obj in data:
                        if isinstance(obj, dict):
                            if 'Reviews' in obj:
                                reviews.extend(obj['Reviews'])
                            elif 'text' in obj or 'review_text' in obj:
                                reviews.append(obj)
                    print(f"Found {len(reviews)} total reviews")
                elif isinstance(data, dict) and ('text' in data or 'review_text' in data):
                    reviews = [data]

                if not reviews:
                    print("No reviews found in JSON structure!")
                    return []

                potential_text_fields = ['text', 'Content', 'ReviewText', 'Text', 'review_text', 'comment', 'content', 'review']
                potential_id_fields = ['review_id', 'ReviewID', 'ReviewId', 'id', 'Id', 'ID', 'reviewId', 'Review_ID']

                review_field_name = None
                review_id_field = None

                for field in potential_text_fields:
                    if field in reviews[0]:
                        review_field_name = field
                        print(f"Found review text in field: '{field}'")
                        break

                for field in potential_id_fields:
                    if field in reviews[0]:
                        review_id_field = field
                        print(f"Found review ID in field: '{field}'")
                        break

                if not review_field_name:
                    print("Could not identify review text field. Available fields:")
                    print(list(reviews[0].keys()) if reviews else "No reviews found")
                    return []

                for i, review in enumerate(reviews):
                    if i >= 1000:  # Limit for efficiency
                        break

                    total_reviews += 1
                    text = review.get(review_field_name, '')

                    if not text or not isinstance(text, str):
                        continue

                    processed_tokens = preprocess_text(text)

                    if processed_tokens:
                        if review_id_field and review.get(review_id_field):
                            review_id = str(review.get(review_id_field))
                        else:
                            review_id = f"review_{i}"

                        documents.append({
                            'review_id': review_id,
                            'text': text,
                            'processed_text': ' '.join(processed_tokens),
                            'tokens': processed_tokens,
                            'stars': review.get('stars', 0),
                            'business_id': review.get('business_id', '')
                        })

            except json.JSONDecodeError:
                print("Single JSON parsing failed, trying line-by-line parsing...")
                f.seek(0)
                for line_num, line in enumerate(f):
                    if line_num >= 1000:
                        break
                    try:
                        line = line.strip()
                        if not line:
                            continue
                        review = json.loads(line)

                        text = review.get('text', '')
                        if not text:
                            continue

                        processed_tokens = preprocess_text(text)

                        if processed_tokens:
                            documents.append({
                                'review_id': review.get('review_id', f'review_{line_num}'),
                                'text': text,
                                'processed_text': ' '.join(processed_tokens),
                                'tokens': processed_tokens,
                                'stars': review.get('stars', 0),
                                'business_id': review.get('business_id', '')
                            })
                        total_reviews += 1

                    except json.JSONDecodeError:
                        continue
                    except Exception as e:
                        print(f"Skipping line {line_num}: {e}")
                        continue

        print(f"✓ Processed {total_reviews} reviews")
        print(f"✓ Loaded {len(documents)} valid documents with preprocessing")
        return documents

    except FileNotFoundError:
        print(f"✗ File not found: {file_path}")
        return []
    except Exception as e:
        print(f"✗ Error loading data: {e}")
        return []

class AdvancedLLMRetriever:
    """Advanced LLM-based document retrieval with pointwise, pairwise, and listwise approaches"""

    def __init__(self, api_key: str = None, model_name: str = CHOSEN_LLM_MODEL):
        self.client = client
        self.model_name = model_name
        self.request_delay = 2  # Increased delay for more API calls

    def get_candidates_tfidf(self, query: str, documents: List[Dict], top_k: int = 5) -> Tuple[List[str], List[str], List[float]]:
        """Get candidate documents using TF-IDF for initial filtering"""
        print(f"Getting TF-IDF candidates for query: '{query}'")

        query_tokens = preprocess_text(query)
        processed_query = ' '.join(query_tokens)
        print(f"Processed query: '{processed_query}'")

        processed_texts = [doc['processed_text'] for doc in documents]
        doc_ids = [doc['review_id'] for doc in documents]

        vectorizer = TfidfVectorizer(
            stop_words=None,
            max_features=1000,
            ngram_range=(1, 2),
            min_df=2,
            lowercase=False
        )

        try:
            doc_vectors = vectorizer.fit_transform(processed_texts)
            query_vector = vectorizer.transform([processed_query])

            similarities = cosine_similarity(query_vector, doc_vectors).flatten()
            top_indices = np.argsort(similarities)[::-1][:top_k]

            candidate_docs = [documents[i]['text'] for i in top_indices]
            candidate_ids = [doc_ids[i] for i in top_indices]
            candidate_scores = [similarities[i] for i in top_indices]

            print(f"Selected {len(candidate_docs)} candidates")
            top_sims = [f"{similarities[i]:.4f}" for i in top_indices[:3]]
            print(f"Top similarities: {top_sims}")
            return candidate_ids, candidate_docs, candidate_scores

        except Exception as e:
            print(f"TF-IDF error: {e}, falling back to first {top_k} documents")
            return (doc_ids[:top_k],
                   [doc['text'] for doc in documents[:top_k]],
                   [0.0] * top_k)

    def pointwise_rerank(self, query: str, candidate_docs: List[str], candidate_ids: List[str]) -> List[Tuple[str, float]]:
        """
        Pointwise approach: Score each document individually for relevance to the query
        Returns: List of (doc_id, relevance_score) tuples
        """
        print(f"Pointwise re-ranking for query: '{query}'")

        scored_docs = []

        for i, (doc_id, doc_text) in enumerate(zip(candidate_ids, candidate_docs)):
            truncated_doc = doc_text[:400] + "..." if len(doc_text) > 400 else doc_text

            prompt = f"""Rate the relevance of this restaurant review to the query "{query}" on a scale of 1-10 (where 10 is highly relevant and 1 is not relevant).

Review: {truncated_doc}

Provide only a single number (1-10) as your response:"""

            try:
                time.sleep(self.request_delay)

                response = self.client.chat.completions.create(
                    model=self.model_name,
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=10,
                    temperature=0.1
                )

                score_text = response.choices[0].message.content.strip()

                # Extract score
                score_match = re.search(r'\b([1-9]|10)\b', score_text)
                if score_match:
                    score = float(score_match.group(1))
                else:
                    score = 5.0  # Default middle score

                scored_docs.append((doc_id, score))
                print(f"  Doc {i+1}: Score {score}")

            except Exception as e:
                print(f"Pointwise error for doc {i+1}: {e}")
                scored_docs.append((doc_id, 5.0))  # Default score

        # Sort by score descending
        scored_docs.sort(key=lambda x: x[1], reverse=True)
        return scored_docs

    def pairwise_rerank(self, query: str, candidate_docs: List[str], candidate_ids: List[str]) -> List[str]:
        """
        Pairwise approach: Compare documents in pairs to determine relative ranking
        Returns: List of doc_ids in ranked order
        """
        print(f"Pairwise re-ranking for query: '{query}'")

        n_docs = len(candidate_docs)
        if n_docs < 2:
            return candidate_ids

        # Create pairwise comparison matrix
        wins = {doc_id: 0 for doc_id in candidate_ids}
        comparisons = 0

        # Compare each pair of documents
        for i in range(n_docs):
            for j in range(i + 1, n_docs):
                doc1_id, doc1_text = candidate_ids[i], candidate_docs[i]
                doc2_id, doc2_text = candidate_ids[j], candidate_docs[j]

                # Truncate documents
                doc1_truncated = doc1_text[:300] + "..." if len(doc1_text) > 300 else doc1_text
                doc2_truncated = doc2_text[:300] + "..." if len(doc2_text) > 300 else doc2_text

                prompt = f"""Given the query "{query}", which of these two restaurant reviews is more relevant?

Review A: {doc1_truncated}

Review B: {doc2_truncated}

Respond with only "A" or "B":"""

                try:
                    time.sleep(self.request_delay)

                    response = self.client.chat.completions.create(
                        model=self.model_name,
                        messages=[{"role": "user", "content": prompt}],
                        max_tokens=5,
                        temperature=0.1
                    )

                    choice = response.choices[0].message.content.strip().upper()

                    if choice == 'A':
                        wins[doc1_id] += 1
                        print(f"  {doc1_id} beats {doc2_id}")
                    elif choice == 'B':
                        wins[doc2_id] += 1
                        print(f"  {doc2_id} beats {doc1_id}")
                    else:
                        # Tie or unclear response
                        wins[doc1_id] += 0.5
                        wins[doc2_id] += 0.5
                        print(f"  Tie between {doc1_id} and {doc2_id}")

                    comparisons += 1

                except Exception as e:
                    print(f"Pairwise comparison error: {e}")
                    # Default to no preference
                    wins[doc1_id] += 0.5
                    wins[doc2_id] += 0.5

        # Sort by number of wins
        ranked_docs = sorted(wins.items(), key=lambda x: x[1], reverse=True)
        return [doc_id for doc_id, _ in ranked_docs]

    def listwise_rerank(self, query: str, candidate_docs: List[str], candidate_ids: List[str]) -> List[str]:
        """
        Listwise approach: Rank all documents simultaneously
        Returns: List of doc_ids in ranked order
        """
        print(f"Listwise re-ranking for query: '{query}'")

        # Build document list for prompt
        doc_list = ""
        for i, (doc_id, doc_text) in enumerate(zip(candidate_ids, candidate_docs)):
            truncated_doc = doc_text[:250] + "..." if len(doc_text) > 250 else doc_text
            doc_list += f"{i+1}. (ID: {doc_id}) {truncated_doc}\n\n"

        prompt = f"""Rank the following restaurant reviews from most relevant to least relevant for the query: "{query}"

{doc_list}

Provide your ranking as a comma-separated list of numbers (e.g., "3,1,5,2,4"):"""

        try:
            time.sleep(self.request_delay)

            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=50,
                temperature=0.1
            )

            ranking_text = response.choices[0].message.content.strip()
            print(f"LLM ranking response: {ranking_text}")

            # Parse ranking
            numbers = re.findall(r'\b([1-9])\b', ranking_text)

            if len(numbers) >= len(candidate_ids):
                # Map numbers to document IDs
                ranked_ids = []
                for num_str in numbers[:len(candidate_ids)]:
                    try:
                        idx = int(num_str) - 1  # Convert to 0-based
                        if 0 <= idx < len(candidate_ids):
                            if candidate_ids[idx] not in ranked_ids:
                                ranked_ids.append(candidate_ids[idx])
                    except ValueError:
                        continue

                # Add any missing documents
                for doc_id in candidate_ids:
                    if doc_id not in ranked_ids:
                        ranked_ids.append(doc_id)

                return ranked_ids[:len(candidate_ids)]
            else:
                print("Failed to parse ranking, using original order")
                return candidate_ids

        except Exception as e:
            print(f"Listwise ranking error: {e}")
            return candidate_ids

def run_advanced_retrieval_experiment():
    """Run advanced LLM retrieval experiment with multiple approaches"""

    if not test_api_connection():
        print("Cannot proceed without API connection.")
        return None, None

    # Load data
    reviews_data = load_yelp_data("/content/sample_data/yelp")

    if not reviews_data:
        print("No data loaded. Cannot proceed with experiment.")
        return None, None

    print(f"Loaded {len(reviews_data)} preprocessed documents")

    # Test queries (using subset to manage API calls)
    test_queries = [
        "fried chicken",
        "BBQ sandwiches",
        "mashed potatoes",
        "FRIENDLY STAFF",
        "Grilled Cheese"
    ]

    retriever = AdvancedLLMRetriever()
    results = {}

    print(f"\n=== Running Advanced LLM Retrieval Experiment ===")
    print(f"Testing {len(test_queries)} queries with 3 advanced approaches")

    for query in test_queries:
        print(f"\n{'='*60}")
        print(f"Processing Query: '{query}'")
        print(f"{'='*60}")

        # Get TF-IDF candidates
        candidate_ids, candidate_docs, tfidf_scores = retriever.get_candidates_tfidf(
            query, reviews_data, top_k=5
        )

        results[query] = {
            'tfidf_baseline': candidate_ids[:3],
            'tfidf_scores': tfidf_scores[:3]
        }

        # Test each advanced approach
        approaches = ['pointwise', 'pairwise', 'listwise']

        for approach in approaches:
            print(f"\n--- {approach.upper()} Approach ---")
            try:
                if approach == 'pointwise':
                    scored_docs = retriever.pointwise_rerank(query, candidate_docs, candidate_ids)
                    ranked_ids = [doc_id for doc_id, score in scored_docs[:3]]
                    results[query][approach] = ranked_ids
                    results[query][f'{approach}_scores'] = [score for doc_id, score in scored_docs[:3]]

                elif approach == 'pairwise':
                    ranked_ids = retriever.pairwise_rerank(query, candidate_docs, candidate_ids)
                    results[query][approach] = ranked_ids[:3]

                elif approach == 'listwise':
                    ranked_ids = retriever.listwise_rerank(query, candidate_docs, candidate_ids)
                    results[query][approach] = ranked_ids[:3]

                print(f"{approach.capitalize()} results: {results[query][approach]}")

            except Exception as e:
                print(f"Error with {approach}: {e}")
                results[query][approach] = candidate_ids[:3]  # Fallback

    return results, reviews_data

def analyze_advanced_results(results: Dict, reviews_data: List[Dict]):
    """Analyze and compare advanced retrieval results"""
    print(f"\n{'='*80}")
    print("ADVANCED LLM RETRIEVAL ANALYSIS")
    print(f"{'='*80}")

    doc_lookup = {doc['review_id']: doc for doc in reviews_data}

    for query, query_results in results.items():
        print(f"\n🎯 Query: '{query}'")
        print("-" * 60)

        approaches = ['tfidf_baseline', 'pointwise', 'pairwise', 'listwise']

        for approach in approaches:
            if approach in query_results:
                print(f"\n📊 {approach.upper()} Results:")
                doc_ids = query_results[approach]

                if f'{approach}_scores' in query_results:
                    scores = query_results[f'{approach}_scores']
                else:
                    scores = [None] * len(doc_ids)

                for i, (doc_id, score) in enumerate(zip(doc_ids, scores), 1):
                    if doc_id in doc_lookup:
                        doc = doc_lookup[doc_id]
                        preview = doc['text'][:120] + "..." if len(doc['text']) > 120 else doc['text']
                        score_str = f" (Score: {score:.1f})" if score is not None else ""
                        print(f"  {i}. {doc_id}{score_str}")
                        print(f"     {preview}")
                        print(f"     Stars: {doc.get('stars', 'N/A')}")
                    else:
                        print(f"  {i}. {doc_id} (not found)")

        # Compare approach overlaps
        print(f"\n🔄 Approach Overlaps for '{query}':")
        for i, approach1 in enumerate(approaches):
            for approach2 in approaches[i+1:]:
                if approach1 in query_results and approach2 in query_results:
                    docs1 = set(query_results[approach1])
                    docs2 = set(query_results[approach2])
                    overlap = len(docs1.intersection(docs2))
                    print(f"  {approach1} vs {approach2}: {overlap}/3 documents overlap")

def save_advanced_results(results: Dict, reviews_data: List[Dict]):
    """Save advanced retrieval results"""
    output_data = {
        'advanced_retrieval_results': results,
        'metadata': {
            'total_documents': len(reviews_data),
            'queries_tested': len(results),
            'approaches_used': ['tfidf_baseline', 'pointwise', 'pairwise', 'listwise'],
            'model_used': CHOSEN_LLM_MODEL,
            'description': 'Advanced LLM retrieval comparison: pointwise, pairwise, and listwise approaches'
        }
    }

    with open('advanced_llm_retrieval_results.json', 'w') as f:
        json.dump(output_data, f, indent=2)
    print(f"\nAdvanced results saved to 'advanced_llm_retrieval_results.json'")

# Main execution
if __name__ == "__main__":
    print("\n🚀 Starting Advanced LLM Retrieval Experiment")
    print("Testing Pointwise, Pairwise, and Listwise approaches")

    # Run experiment
    experiment_results, reviews_data = run_advanced_retrieval_experiment()

    if experiment_results and reviews_data:
        # Analyze results
        analyze_advanced_results(experiment_results, reviews_data)

        # Save results
        save_advanced_results(experiment_results, reviews_data)

        print(f"\n✅ Advanced Experiment Complete!")
        print(f"✓ Tested 3 advanced LLM approaches")
        print(f"✓ Compared against TF-IDF baseline")
        print(f"✓ Analyzed relevance and overlap patterns")

        # Summary insights
        print(f"\n💡 Key Insights:")
        print("- Pointwise: Individual relevance scoring for each document")
        print("- Pairwise: Head-to-head comparisons between document pairs")
        print("- Listwise: Simultaneous ranking of all candidates")
        print("- Compare overlap patterns to see which approaches agree/disagree")

    else:
        print("❌ Advanced experiment failed. Check API connection and data.")

Advanced LLM Retrieval: Pointwise, Pairwise, and Listwise Approaches

🚀 Starting Advanced LLM Retrieval Experiment
Testing Pointwise, Pairwise, and Listwise approaches

=== Testing API Connection ===
✓ API Connection Successful!
Test Response: 

=== Loading Yelp Data from /content/sample_data/yelp ===
Path is a directory. Searching for review files...

=== Scanning directory: /content/sample_data/yelp ===
Found files: ['/content/sample_data/yelp/dlDEuDIvZI6I0cGZy4jIYg.json', '/content/sample_data/yelp/okChDSotPCRtJlQVzlnw1Q.json', '/content/sample_data/yelp/pVEyB1BxiZkJUeoqHG3ehA.json']...
Using file: /content/sample_data/yelp/dlDEuDIvZI6I0cGZy4jIYg.json
Found 1045 reviews in a single JSON object
Found review text in field: 'Content'
Found review ID in field: 'ReviewID'
✓ Processed 1000 reviews
✓ Loaded 1000 valid documents with preprocessing
Loaded 1000 preprocessed documents

=== Running Advanced LLM Retrieval Experiment ===
Testing 5 queries with 3 advanced approaches

Processing Qu

# Extra Credits (10pts)

Further explore LLM-based retrieval. For example, you may consider pointwise, pairwise, and/or listwise prompts. Retrieve the top 3 documents for each query. Did you find the documents more relevant than documents retrieved by ?


# Submission

This assignment has in total 100 points. The deadline is June 4th 23:59 PDT. You should submit your report in **PDF** using the homework latex template, and submit your code (notebook).